## Competition link:<a href="https://www.kaggle.com/competitions/mercor-ai-detection/overview">Mercor AI Text Detection</a>
## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a> 2025/10/09

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">1.Import Libraries</h1></span>

Due to the continuous updates of the <a href="https://github.com/yunsuxiaozi/Yunbase">Yunbase</a> GitHub repository, a static version is used <a href="https://www.kaggle.com/code/yunsuxiaozi/yunbase">here</a>.We are using version 24 here.

This is a framework I developed for the convenience of playing competitions and avoiding writing duplicate code. You can simply understand it as automl, although its current performance is not as good as automl.

If you are not familiar with `Yunbase`, it's okay. Yunbase mainly encapsulates some commonly used operations such as `onehotencoder`、`kfold`, You can also manually implement the same function without using Yunbase. This notebook mainly provides beginners with some ideas for feature engineering and cross validation.

If there is a long time, such as a month or even a year, after the release date of this notebook, this code may not be able to run directly.Since the version of `Yunbase` will be constantly updated, if you want to run this code successfully, you might first download the output from the corresponding version in <a href="https://www.kaggle.com/code/yunsuxiaozi/yunbase">Yunbase</a>, save it as your own dataset, and then import it.

## About pytabkit

Yunbase currently supports five models: lightgbm, xgboost, catboost, realmlp, and tabm. It can also support other models that provide fit and predict interfaces, such as linear regression, but other models may have compatibility issues.

The use of realmlp and tabm can be done using the pytabkit library, which provides interfaces for fit and predict.

In [1]:
!pip -q install -U pytabkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cur

In [2]:
import sys
sys.path.append('/kaggle/input/yunbase/Yunbase')

In [3]:
!pip install -q --requirement /kaggle/input/yunbase/Yunbase/requirements.txt  \
--no-index --find-links file:/kaggle/input/yunbase/

  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytabkit 1.6.1 requires scikit-learn>=1.3, but you have scikit-learn 1.2.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.0 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.2.2 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn

In [4]:
from baseline import Yunbase
import pandas as pd#read csv,parquet
import numpy as np#for scientific computation of matrices

from  lightgbm import LGBMRegressor,LGBMClassifier,log_evaluation,early_stopping
from catboost import CatBoostRegressor,CatBoostClassifier
from pytabkit import RealMLP_TD_Classifier,RealMLP_TD_Regressor
from xgboost import XGBRegressor,XGBClassifier
from pytabkit import TabM_D_Classifier,TabM_D_Regressor

#gene(topic) similarity   
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer#word2vec feature

import warnings#avoid some negligible errors
#The filterwarnings () method is used to set warning filters, which can control the output method and level of warning information.
warnings.filterwarnings('ignore')
import random#provide some function to generate random_seed.
#set random seed,to make sure model can be recurrented.
def seed_everything(seed):
    np.random.seed(seed)#numpy's random seed
    random.seed(seed)#python built-in random seed
seed_everything(seed=2025)

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">2.Load dataset</h1></span>

We see that the train and test data for this competition are very small, so we will use simple feature engineering to prevent overfitting.

In [5]:
target_col='is_cheating'
train=pd.read_csv("/kaggle/input/mercor-ai-detection/train.csv")
test=pd.read_csv("/kaggle/input/mercor-ai-detection/test.csv")
print(f"train.shape:{train.shape},test.shape:{test.shape}")
train.head()

train.shape:(269, 4),test.shape:(264, 3)


,id,topic,answer,is_cheating
0,form_r_d5TpupthGXvfy58LDIbkqp,A girl wakes from a dream and she is not sure ...,"My eyes flew open, and the air around me feels...",1
1,form_r_AAABmK2rZBBbtfdHxzhBY4C4,A journalistic review piece about the top 6 ai...,Robot Butlers in the year of 2025. What are th...,0
2,form_r_Cz6IJIWUj1B7pdyO6zPhGF,The influence of fictional universities in cam...,"In recent years, apparel featuring the names a...",1
3,form_r_AAABmKyCfdO6NLv8eqBPCYD_,Why do girls love horses,"The moment before I hit the dirt, I thought we...",0
4,form_r_WS8I1NJpwvIcFL3Xv0Qhuc,"Every year, a remote mountain town elects a ne...","In the valley of Eldermist, were the mountains...",1


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">3.Feature Engineer</h1></span>


There are mainly two methods used here to represent text as word vectors. Since Yunbase provides implementation methods internally, all that is needed here is to know which model was used, which column of text was processed, and finally write the name of the model.

In [6]:
word2vec_models=[(TfidfVectorizer(analyzer='char',max_features=256,ngram_range=(1,3)),'topic','tfidf'),
                 (TfidfVectorizer(analyzer='char',max_features=256,ngram_range=(1,3)),'answer','tfidf'),
                 (Word2Vec(vector_size=128, window=3, min_count=1, workers=16),'topic','word2vec'),
                 (Word2Vec(vector_size=128, window=4, min_count=1, workers=16),'answer','word2vec'),
                ]

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">4.Model training</h1></span>

For such small datasets, a common approach is to simply perform feature engineering and add a large number of models without parameter tuning.If you tune the model, you may get a model that overfits the training data.

In [7]:
models=[ (LGBMClassifier(),'lgb'),
         (CatBoostClassifier(),'cat'),
         (TabM_D_Classifier(weight_decay=1e-4),'tabm'),
         (RealMLP_TD_Classifier(wd=1e-4),'realmlp'),
         (XGBClassifier(enable_categorical=True),'xgb')
       ] 

In [8]:
yunbase_params={'objective':'binary','metric':'auc', 'num_classes':2,
                'target_col':target_col,'drop_cols':['id'],'seed':2025,
                'models':models,'early_stop':1024,'word2vec_models':word2vec_models,
               }
yunbase=Yunbase(**yunbase_params)
yunbase.fit(train,save_trained_models=False)

Currently supported metrics:['custom_metric', 'mae', 'rmse', 'mse', 'medae', 'rmsle', 'msle', 'mape', 'r2', 'smape', 'auc', 'pr_auc', 'logloss', 'f1_score', 'mcc', 'accuracy', 'multi_logloss']
Currently supported models:['lgb', 'cat', 'xgb', 'ridge', 'Lasso', 'LinearRegression', 'LogisticRegression', 'tabnet', 'realmlp(pytabkit need install yourself)', 'tabm(pytabkit need install yourself)', 'Word2Vec', 'tfidfvec', 'countvec']
Currently supported kfolds:['KFold', 'GroupKFold', 'StratifiedKFold', 'StratifiedGroupKFold', 'purged_CV', 'custom_kfold']
Currently supported objectives:['binary', 'multi_class', 'regression']
fit......
load train data
Feature Engineer


  0%|          | 0/2 [00:00<?, ?it/s]

-> for column answer text clean


Pandas Apply:   0%|          | 0/269 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:00<00:00,  1.76it/s]

...

name:xgb,fold:3
< word2vec >
-> for column topic tfidf word2vec feature
-> for column answer tfidf word2vec feature
-> for column topic word2vec word2vec feature
-> for column answer word2vec word2vec feature
< word2vec >
-> for column topic tfidf word2vec feature
-> for column answer tfidf word2vec feature
-> for column topic word2vec word2vec feature
-> for column answer word2vec word2vec feature
[0]	validation_0-logloss:0.57349	validation_0-auc:0.82276
[99]	validation_0-logloss:0.30061	validation_0-auc:0.94069


 80%|████████  | 4/5 [00:14<00:03,  3.63s/it]

name:xgb,fold:4
< word2vec >
-> for column topic tfidf word2vec feature
-> for column answer tfidf word2vec feature
-> for column topic word2vec word2vec feature
-> for column answer word2vec word2vec feature
< word2vec >
-> for column topic tfidf word2vec feature
-> for column answer tfidf word2vec feature
-> for column topic word2vec word2vec feature
-> for column answer word2vec word2vec feature
[0]	validation_0-logloss:0.58046	validation_0-auc:0.83333
[99]	validation_0-logloss:0.28460	validation_0-auc:0.95115


100%|██████████| 5/5 [00:18<00:00,  3.64s/it]

auc------------------------------>0.9297423887587822


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">5.Create submission</h1></span>

Predict the test data and save the submission.csv. Here, weighted ensemble can be performed based on the results of cross validation.

In [9]:
#0.918813	0.931638	0.905654	0.889651	0.933311
test_preds=yunbase.predict(test,weights=[2,3,2,1,3])
yunbase.submit("/kaggle/input/mercor-ai-detection/sample_submission.csv",test_preds=test_preds)

predict......
weight normalization
weights:[0.90909094 1.3636364  0.90909094 0.45454547 1.3636364 ]
final_repeat0_auc:0.9431247908999666
load test data
Feature Engineer


  0%|          | 0/2 [00:00<?, ?it/s]

-> for column answer text clean


Pandas Apply:   0%|          | 0/264 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:00<00:00,  1.86it/s]

-> for column topic text clean


Pandas Apply:   0%|          | 0/264 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  3.10it/s]


< one hot encoder >
< drop useless cols >
nan_cols:[]
unique_cols:[]
drop_cols:['id']
high_corr_cols:[]
------------------------------
test.shape:(264, 2)
prediction on test data
< word2vec >
-> for column topic tfidf word2vec feature
-> for column answer tfidf word2vec feature
-> for column topic word2vec word2vec feature
-> for column answer word2vec word2vec feature
< word2vec >
-> for column topic tfidf word2vec feature
-> for column answer tfidf word2vec feature
-> for column topic word2vec word2vec feature
-> for column answer word2vec word2vec feature
< word2vec >
-> for column topic tfidf word2vec feature
-> for column answer tfidf word2vec feature
-> for column topic word2vec word2vec feature
-> for column answer word2vec word2vec feature
< word2vec >
-> for column topic tfidf word2vec feature
-> for column answer tfidf word2vec feature
-> for column topic word2vec word2vec feature
-> for column answer word2vec word2vec feature
< word2vec >
-> for column topic tfidf word2vec f

In [10]:
display(yunbase.CVMetricsSummary())

,lgb,cat,tabm,realmlp,xgb
auc,0.920096,0.934594,0.881956,0.894056,0.929742
pr_auc,0.939185,0.955524,0.894168,0.913589,0.947838
logloss,0.358430,0.331954,0.461885,0.414478,0.347532
f1_score,0.861953,0.868421,0.837209,0.844291,0.863787
mcc,0.692043,0.699904,0.631591,0.664148,0.691967
accuracy,0.847584,0.851301,0.817844,0.832714,0.847584
multi_logloss,0.358430,0.331954,0.461885,0.414478,0.347532
